In [23]:
# mounts Google Drive to the Colab VM
from google.colab import drive, files
drive.mount('/content/drive')

# project github foldername
FOLDERNAME = '/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/'

# load python files from foldername
import sys
sys.path.append(FOLDERNAME)

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [288]:
import xarray as xr
import pandas as pd
import pickle
import datetime 
import numpy as np
import os 
from os import path

!sudo apt-get install libeccodes0

!pip install cfgrib
!pip install netcdf4

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libeccodes0 is already the newest version (2.6.0-2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [0]:
directory = "/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/"
date = datetime.datetime(2018, 7, 16, 0,0)

level = 'P_sea_level'
model = 'arpege'
zone = "NW"
directory_2 = directory + 'models_2D_' + zone + '/'

fname= directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.grib'
!mv $fname '/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/Wrong_values'


In [293]:
directory = "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/"
date = datetime.datetime(2017, 12, 05, 0,0)

level = '2m'
model = 'arpege'
zone = "NW"
directory_2 = directory + 'models_2D_' + zone + '/'


data = create_missing_array(directory, zone, model, level, date)

2017-10-26 00:00:00
0.6666666666666666
0.3333333333333333


In [0]:
def create_missing_array(directory, zone, model, level, date):
  if level == '2m':
    params = ['t2m', 'd2m', 'r']
  elif level == '10m':
    params = ['ws', 'p3031', 'u10', 'v10']
  elif level == 'P_sea_level':
    params = ['msl']
  else:
    params = ['tp']

  date_prev = date + datetime.timedelta(days = -1)
  fname_prev = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_prev.year}{str(date_prev.month).zfill(2)}{str(date_prev.day).zfill(2)}000000.grib'
  days_prev = 1
  while path.exists(fname_prev) == False:
    date_prev = date_prev + datetime.timedelta(days = -1)
    print(date_prev)
    fname_prev = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_prev.year}{str(date_prev.month).zfill(2)}{str(date_prev.day).zfill(2)}000000.grib'
    days_prev += 1
  data_prev = xr.open_dataset(fname_prev, engine='cfgrib') 

  days_next = 1
  date_next = date + datetime.timedelta(days = 1)
  fname_next = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_next.year}{str(date_next.month).zfill(2)}{str(date_next.day).zfill(2)}000000.grib'
  while path.exists(fname_next) == False:
    date_next = date_next + datetime.timedelta(days = 1)
    print(date_next)
    fname_next = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_next.year}{str(date_next.month).zfill(2)}{str(date_next.day).zfill(2)}000000.grib'
    days_next +=1
  data_next = xr.open_dataset(fname_next, engine='cfgrib') 

  if level !=  'PRECIP':
    if data_prev.step.values.shape[0] != 25:
      data_prev = interpolate_in_time(data_prev, 25)
  else:
    if data_prev.step.values.shape[0] !=24:
      data_prev = interpolate_in_time(data_prev, 24)
  if level !=  'PRECIP':
    if data_next.step.values.shape[0] != 25:
      data_next = interpolate_in_time(data_next, 25)
  else:
    if data_next.step.values.shape[0] !=24:
      data_next = interpolate_in_time(data_next, 24)


  data = data_prev.copy()
  for param in params:
    data[param] = days_next/(days_prev + days_next)*data_prev[param] + days_prev/(days_prev + days_next)*data_next[param]
  print(days_next/(days_prev + days_next))
  print(days_prev/(days_prev + days_next))


  data = data.assign_coords({'time' : date})
  data = data.assign_coords({'valid_time': data['valid_time'].values + np.timedelta64(days_prev,'D')})

  return data
  # !mv $fname '/content/d

In [0]:
x = xr.Dataset(
     {
         "t2m": (
             ("step","latitude", "longitude"),
             param_values,
         ),
         "precipitation": (("latitude", "longitude"), np.random.rand(58*80).reshape(58,80)),
     },
     coords={"latitude": lala_27["latitude"], "longitude": lala_27["longitude"],
             "step": np.array(valid_time_values-valid_time_values[0], dtype='timedelta64[ns]'),
             "valid_time": valid_time_values, "time": lala_27["time"]}
)

In [8]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [0]:
lala = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170826000000.nc')

In [0]:
lala_25 = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170825000000.grib', engine='cfgrib')
lala_27 = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170827000000.grib', engine='cfgrib')

In [0]:
def interpolate_in_time(array, num_timesteps):
  T = array.dims['step']
  H = array.dims['latitude']
  W = array.dims['longitude']
  array_times = array['valid_time'].values
  date = array['time'].values
  array_times_pd = pd.to_datetime(array_times)
  date_pd = pd.to_datetime(date)
  hours = list(array_times_pd.hour)
  if array_times_pd[-1].day > date_pd.day:
    hours[-1] = num_timesteps
  final_var = {}
  for var in array.data_vars:
    new_array = np.zeros([num_timesteps, H, W])
    for h in range(len(hours)):
      new_array[hours[h]] = array[var][h]
    for h in range(num_timesteps):
      if h not in hours:
        if h < hours[0]:
          new_array[h] = array[var][hours[0]]
        elif h > hours[-1]:
          new_array[h] = array[var][hours[-1]]
        else:
          m = np.argmin(abs(np.array(hours) - h))
          if hours[m] < h:
            new_array[h] = (hours[m+1] - h)/(hours[m + 1]- hours[m])*array[var][m] + (h-hours[m])/(hours[m+1]-hours[m])*array[var][m+1]
          else:
            new_array[h] = (hours[m] - h)/(hours[m]- hours[m-1])*array[var][m-1] + (h-hours[m-1])/(hours[m]-hours[m-1])*array[var][m]
    final_var[var] = (("step", "latitude", "longitude"), new_array)
  new_dates = np.array(pd.date_range(date_pd, periods=num_timesteps, freq='H'))
  new_xarray = xr.Dataset(
     final_var, coords={"latitude": array["latitude"], "longitude": array["longitude"],
             "step": np.array(new_dates-new_dates[0], dtype='timedelta64[ns]'),
             "valid_time": new_dates, "time": array["time"]}
)
  return new_xarray

                        

In [269]:
new_lala_27 = interpolate_in_time(lala_27, 25)

[0, 3, 4, 5, 7, 8, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 23, 24]


In [270]:
new_lala_27

<xarray.Dataset>
Dimensions:            (latitude: 58, longitude: 80, step: 25, valid_time: 25)
Coordinates:
    time               datetime64[ns] 2017-08-27
    heightAboveGround  int64 2
  * latitude           (latitude) float64 51.9 51.8 51.7 51.6 ... 46.4 46.3 46.2
  * longitude          (longitude) float64 -5.842 -5.742 -5.642 ... 1.958 2.058
  * step               (step) timedelta64[ns] 00:00:00 ... 1 days 00:00:00
  * valid_time         (valid_time) datetime64[ns] 2017-08-27 ... 2017-08-28
Data variables:
    t2m                (step, latitude, longitude) float64 288.3 288.1 ... 294.8
    d2m                (step, latitude, longitude) float64 287.5 287.4 ... 289.6
    r                  (step, latitude, longitude) float64 95.26 96.07 ... 72.04

In [271]:
lala_27

<xarray.Dataset>
Dimensions:            (latitude: 58, longitude: 80, step: 18)
Coordinates:
    time               datetime64[ns] 2017-08-27
  * step               (step) timedelta64[ns] 00:00:00 ... 1 days 00:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 51.9 51.8 51.7 51.6 ... 46.4 46.3 46.2
  * longitude          (longitude) float64 -5.842 -5.742 -5.642 ... 1.958 2.058
    valid_time         (step) datetime64[ns] 2017-08-27 ... 2017-08-28
Data variables:
    t2m                (step, latitude, longitude) float32 ...
    d2m                (step, latitude, longitude) float32 ...
    r                  (step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2020-06-04T19:46:52 GRIB to CDM+CF via cfgrib-0....

In [0]:
new_fname = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'

In [0]:
new_fname = '/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170826000000.nc'
!rm $new_fname
# new_fname = f'/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_{zone}/{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'

# data.to_netcdf(new_fname, engine = 'netcdf4')

In [0]:
!wget "https://meteonet.umr-cnrm.fr/dataset/data/NW/weather_models/2D_parameters/NW_weather_models_2D_parameters_2017.tar.gz"

--2020-06-03 20:13:40--  https://meteonet.umr-cnrm.fr/dataset/data/NW/weather_models/2D_parameters/NW_weather_models_2D_parameters_2017.tar.gz
Resolving meteonet.umr-cnrm.fr (meteonet.umr-cnrm.fr)... 193.49.97.131
Connecting to meteonet.umr-cnrm.fr (meteonet.umr-cnrm.fr)|193.49.97.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8789487640 (8.2G) [application/x-gzip]
Saving to: ‘NW_weather_models_2D_parameters_2017.tar.gz’

NW_weather_models_2 100%[===================>]   8.19G  10.5MB/s    in 13m 46s 

2020-06-03 20:27:28 (10.1 MB/s) - ‘NW_weather_models_2D_parameters_2017.tar.gz’ saved [8789487640/8789487640]



In [0]:
!mv NW_weather_models_2D_parameters_2017.tar.gz "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/"

In [0]:
!tar -xvzf "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017.tar.gz" -C "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/"

NW_weather_models_2D_parameters_2017/
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201701.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201702.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201703.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201704.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201705.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201706.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201707.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201708.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201709.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201710.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201711.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_m

In [0]:
for i in [4,6,8]:
  print(i)
  if i < 10:
    fname = "/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_20170" + str(i) + ".tar.gz"
  else:
    fname = "/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/NW_rainfall_quality_code_2016/rainfall-diff-quality-code-NW-2016-" + str(i) + ".tar.gz"
  !tar -xzf $fname -C "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017/"

4
6
8
